In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:

import sys

import ipywidgets as widgets

sys.path.append('../')
sys.path.append('../src')

import src.backend
import src.workload_builder as builder
from mbi import Domain
import src.plots
import altair as alt
alt.renderers.enable('default')
from IPython.display import display, clear_output

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


alt.data_transformers.disable_max_rows()

/home/amit/anaconda3/lib/python3.9/site-packages/mbi/__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')
/tmp/ipykernel_21314/218976982.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


DataTransformerRegistry.enable('default')

In [3]:
data_path = '../data/CPS/CPS.csv'
cps_domain = Domain(attrs=('age','income','marital','race'), shape=(100,100,7,4))
questions = ["Warm Up: Type 'Yes' if you are ready to begin!",
            "Question 1/6: How many Chinese and American Indians have 'Widowed' as their marital status?",
            "Question 2/6: How many people over the age of 65 have income between 100,000 and 149,000?",
            "Question 3/6: What is the second most common marital status of people with income in the 100,000 to 199,000 range?",
            "Question 4/6: Choose the top 3 age groups that have marital status: 'Never Married'?",
            "Question 5/6: In the 100,000 to 199,000 income range, rank the top 3 races in terms of count from greatest to least.",
            "Question 6/6: What is the second most common race in the 45-54 age group?"]

marital_race = {'marital':1, 'race':1}
age_income = {'age':10, 'income':10}
marital_income = {'income':10, 'marital':1}
age_marital = {'age':10, 'marital':1}
race_income = {'income':10, 'race':1}
age_race = {'age':10, 'race':1}

visualizations = [None, marital_race, age_income, marital_income, age_marital, race_income, age_race]
prev_spec = None
curr_spec = None
epsilon_increments = [None,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
budget = 1.0
max_tries = 10

index = 0
tries = 0
back_end = None


In [4]:
cps_domain

Domain(age: 100, income: 100, marital: 7, race: 4)

# Logging

In [5]:
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [6]:
plot_output1 = widgets.Output()
plot_output2 = widgets.Output()
plot_output3 = widgets.Output()
plot_output4 = widgets.Output()
plot_output5 = widgets.Output()
plot_output6 = widgets.Output()

In [7]:
plot_output_rm1 = widgets.Output()
plot_output_rm2 = widgets.Output()
plot_output_rm3 = widgets.Output()
plot_output_rm4 = widgets.Output()
plot_output_rm5 = widgets.Output()
plot_output_rm6 = widgets.Output()

In [8]:
plot_output_after = widgets.Output()

In [9]:
plot_output = widgets.Output()

In [10]:
# plot_output

 ## Dropdown Boxes

In [11]:
columns = ['Age','Income','Marital Status']
translate = {'Income': 'income', 'Marital Status': 'marital', 'Age': 'age', 'Race':'race'}

Text Boxes

In [12]:
global index
prompt = widgets.Textarea(
    value=questions[0],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1200px', height='30px')
)

prompt2 = widgets.Textarea(
    value=questions[1],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

prompt3 = widgets.Textarea(
    value=questions[2],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

prompt4 = widgets.Textarea(
    value=questions[3],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

prompt5 = widgets.Textarea(
    value=questions[4],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

prompt6 = widgets.Textarea(
    value=questions[5],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

prompt7 = widgets.Textarea(
    value=questions[6],
    placeholder='Type',
    description='',
    disabled=True,
    layout=widgets.Layout(width='1000px', height='30px')
)

# finished_tries = widgets.Textarea(
#     value="",
#     placeholder='Type',
#     description='',
#     disabled=True,
#     layout=widgets.Layout(width='500px', height='30px') #500,30
# )

answer = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer2 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer3 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer4 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer5 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

answer6 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px', align_items='center')
)

answer7 = widgets.Textarea(
    value='',
    placeholder='',
    description='Answer:',
    layout=widgets.Layout(width='300px', height='40px')
)

In [13]:
def make_plots(idd):
    for i in range(len(questions) - 1):
        if i == idd:
            continue
        # with plot_output_arr[i]:
        column_names = list(visualizations[i+1].keys())
        plot = src.plots.linked_hist(column_names[0], column_names[1], data=current_spec_arr[i].reset_index(column_names), display_true=False, history=False)
        logger.info(plot)
        # print("hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh")

        bar_label = widgets.HBox([budget_spent, progress_bar, remeasure_arr[i]])
        ques = widgets.HBox([prompt_arr[i+1]])
        # prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
        prompt_answer = widgets.VBox([answer_arr[i+1]], layout=box_layout)
        display(bar_label)
        display(ques)

        display(plot)
        histogram_plots.append(widgets.VBox([plot_output_arr[i]]))
        display(histogram_plots[i])
        interface = widgets.VBox([prompt_answer])
        display(interface)


## Buttons

In [14]:
import numpy as np
from ipywidgets import Layout

submit_btn = widgets.Button(description='Start')
submit_btn_final = widgets.Button(description='Submit', layout=Layout(width='20%', height='50px'))
make_it_better1 = widgets.Button(description='Remeasure Q1')
make_it_better2 = widgets.Button(description='Remeasure Q2')
make_it_better3 = widgets.Button(description='Remeasure Q3')
make_it_better4 = widgets.Button(description='Remeasure Q4')
make_it_better5 = widgets.Button(description='Remeasure Q5')
make_it_better6 = widgets.Button(description='Remeasure Q6')
# make_it_better = widgets.Button(description='Remeasure')
# make_it_better = []
# for i in range(len(questions)):
#     make_it_better[i] = widgets.Button(description='Remeasure')

prompt_arr = [prompt, prompt2, prompt3, prompt4, prompt5, prompt6, prompt7]
answer_arr = [answer, answer2, answer3, answer4, answer5, answer6, answer7]
remeasure_arr = [make_it_better1, make_it_better2, make_it_better3, make_it_better4, make_it_better5, make_it_better6]
plot_output_arr = [plot_output1, plot_output2, plot_output3, plot_output4, plot_output5, plot_output6]
plot_output_rm_arr = [plot_output_rm1, plot_output_rm2, plot_output_rm3, plot_output_rm4, plot_output_rm5, plot_output_rm6]

progress_bar = widgets.FloatProgress(min=0.0, max=max_tries) # instantiate the bar
progress_bar.style.bar_color = 'red'
progress_bar.description = str(tries) + '/' + str(max_tries)
budget_spent = widgets.Label(value='Remeasure Used')
num_answers = len(questions)
survey_answers = [0]*num_answers

current_spec_arr = []
hist_workloads = []
histogram_plots = []
# hist_plot = None


def make_vis(do_not_update=None):
    global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end, bar_label, interface, ques
    back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)

    for i in range(len(questions) - 1):
        prompt_arr[i+1].value = questions[i+1]
        hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
        hist_workloads.append(hist)

        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20) # got y_hat

        column_names = list(visualizations[i+1].keys())
        curr_spec, cached = back_end.display(hist)
        current_spec_arr.append(curr_spec)

        with plot_output_arr[i]:
            plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
            logger.info(plot)

        bar_label = widgets.HBox([budget_spent, progress_bar, remeasure_arr[i]])
        ques = widgets.HBox([prompt_arr[i+1]])
        # prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
        prompt_answer = widgets.VBox([answer_arr[i+1]], layout=box_layout)
        display(bar_label)
        display(ques)

        display(plot)
        histogram_plots.append(widgets.VBox([plot_output_arr[i]]))
        display(histogram_plots[i])
        interface = widgets.VBox([prompt_answer])
        display(interface)

def on_click_submit(obj):
    global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end, bar_label, interface, ques, plot, plot_output_arr
    logger.info('Clicked submit')
    if answer.value == '':
        return
    val = answer.value
    answer.value = answer.placeholder
    survey_answers[index] = val

    interface.close()

    make_vis()

    display(submit_btn_final)

    # for i in range(len(questions) - 1):
    #     prompt_arr[i+1].value = questions[i+1]
    #     back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)
    #     hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[i+1])
    #     y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[i+1], restarts=20) # got y_hat
    #     column_names = list(visualizations[i+1].keys())
    #     curr_spec = back_end.display(hist)
    #
    #     with plot_output:
    #         # plot_output.clear_output()
    #         plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
    #         logger.info(plot)
    #
    #     bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
    #     ques = widgets.HBox([prompt_arr[i+1]])
    #     prompt_answer = widgets.VBox([answer_arr[i+1], submit_btn], layout=box_layout)
    #     display(bar_label)
    #     display(ques)
    #     display(plot)
    #     interface = widgets.VBox([plot_output, prompt_answer])
    #     display(interface)

    # with plot_output:
    #     clear_output()
    # submit_btn.close()
    # interface.close()
    # bar_label.close()
    # prompt.value = 'Tutorial finished, thanks for your participation!'
    # display(prompt)
    # return

idx = 1

def remeasure_clicked(question_no):
    global index, visualizations, tries, idx, interface, submit_btn, plot_output_arr, histogram_plots, plot_output_rm_arr
    logger.info('Clicked remeasure, question: {}'.format(index))
    # clear_output()
    plot_output_arr[question_no].clear_output()

    key_val = [x for x in visualizations[question_no].items()]

    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    binning(measure_dict, question_no-1, epsilon = epsilon_increments[question_no])

    # for i in range(len(questions) - 1):
    #     key_val = [x for x in visualizations[i+1].items()]
    #     # logger.info(key_val)
    #     measure_dict = {'left': key_val[0], 'right': key_val[1]}
    #     binning(measure_dict, i, epsilon = epsilon_increments[idx])
    # idx += 1

    # make_vis()
    tries += 1
    progress_bar.value = tries
    progress_bar.description = str(tries) + '/' + str(max_tries)

def on_click_make_it_better(obj):
    print("AAAAA ", obj)
    global index, visualizations, tries, idx, interface, submit_btn, plot_output
    logger.info('Clicked remeasure, question: {}'.format(index))

    with plot_output:
        clear_output()
    # plot_output_before[0].clear_output()
    # print(plot_output_before)
    # submit_btn.close()
    # interface.close()
    # bar_label.close()
    
    key_val = [x for x in visualizations[1].items()]
    print("keyyyyy   ", key_val)
    measure_dict = {'left': key_val[0], 'right': key_val[1]}
    binning(measure_dict, 0, epsilon = epsilon_increments[1])

    # for i in range(len(questions) - 1):
    #     key_val = [x for x in visualizations[i+1].items()]
    #     # logger.info(key_val)
    #     measure_dict = {'left': key_val[0], 'right': key_val[1]}
    #     binning(measure_dict, i, epsilon = epsilon_increments[idx])
    # idx += 1

    # make_vis()
    tries += 1
    progress_bar.value = tries
    progress_bar.description = str(tries) + '/' + str(max_tries)

def on_click_make_it_better_1(obj):
    remeasure_clicked(1)

def on_click_make_it_better_2(obj):
    remeasure_clicked(2)

def on_click_make_it_better_3(obj):
    remeasure_clicked(3)

def on_click_make_it_better_4(obj):
    remeasure_clicked(4)

def on_click_make_it_better_5(obj):
    remeasure_clicked(5)

def on_click_make_it_better_6(obj):
    remeasure_clicked(6)

submit_btn.on_click(on_click_submit)
make_it_better1.on_click(on_click_make_it_better_1)
make_it_better2.on_click(on_click_make_it_better_2)
make_it_better3.on_click(on_click_make_it_better_3)
make_it_better4.on_click(on_click_make_it_better_4)
make_it_better5.on_click(on_click_make_it_better_5)
make_it_better6.on_click(on_click_make_it_better_6)

box_layout = widgets.Layout(display='flex',
    flex_flow='column',
    align_items='flex-start',
    color='black',
    width='80%')

# bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
prompt_answer = widgets.VBox([answer, submit_btn], layout=box_layout)

In [15]:
# import numpy as np
#
# submit_btn = widgets.Button(description='Submit')
# make_it_better = widgets.Button(description='Remeasure')
#
# def on_click_submit(obj):
#     global index, survey_answers, visualizations, epsilon_increments, curr_spec, back_end
#     logger.info('Clicked submit')
#     if answer.value == '':
#         return
#     val = answer.value
#     answer.value = answer.placeholder
#     survey_answers[index] = val
#
#     if index+1 < len(questions):
#         index += 1
#         prompt.value = questions[index]
#     else:
#         with plot_output:
#             clear_output()
#         submit_btn.close()
#         interface.close()
#         bar_label.close()
#         prompt.value = 'Tutorial finished, thanks for your participation!'
#         display(prompt)
#         return
#
#     back_end = src.backend.initialize_backend(cps_domain, data_path, budget=1.0)
#
#     hist = builder.histogram_workload(cps_domain.config, bin_widths=visualizations[index])
#
#     # logger.info(hist.matrix.matrix)
#
#     y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon_increments[index], restarts=20) # got y_hat
#
#     # B = np.linalg.pinv(strategy_matrix)
#
#     # logger.info("printing y_hat")
#     # logger.info(B)
#     logger.info("printing strategy matrix")
# #     logger.info(strategy_matrix.matrix)
#
#     column_names = list(visualizations[index].keys())
#     curr_spec = back_end.display(hist)
#
#     with plot_output:
#         plot_output.clear_output()
#         plot = src.plots.linked_hist(column_names[0], column_names[1], data=curr_spec.reset_index(column_names), display_true=False, history=False)
#         logger.info(plot)
# #         display("Updating...")
#         display(plot)
# #         altair_viewer.show(plot)
#
# def on_click_make_it_better(obj):
#     global index, visualizations, tries
#     logger.info('Clicked remeasure, question: {}'.format(index))
#
#     key_val = [x for x in visualizations[index].items()]
#     logger.info(key_val)
#     measure_dict = {'left': key_val[0], 'right': key_val[1]}
#     binning(measure_dict, epsilon = epsilon_increments[index])
#
# submit_btn.on_click(on_click_submit)
# make_it_better.on_click(on_click_make_it_better)

In [16]:
# text_area = []
# for i in range(len(questions)):
#     text_area.append(answer)


In [17]:
back_end = src.backend.initialize_backend(cps_domain, data_path, budget=budget)
# global back_end

def binning(measure_dict, id=None, epsilon=None):
    global tries, curr_spec, back_end, current_spec_arr, hist_workloads, plot_output_arr, plot_output_rm_arr
    
    left_col = measure_dict['left'] 
    right_col = measure_dict['right']
    
    widths = {left_col[0]:left_col[1], right_col[0]:right_col[1]}
    
    hist = builder.histogram_workload(cps_domain.config, bin_widths=widths)

    if epsilon is not None:
        if tries+1 >= max_tries:
            for i in range(len(questions)-1):
                remeasure_arr[i].close()
                budget_spent.value = 'You have used all the remeasures'
        if tries+1 > max_tries:
            return
        y_hat, strategy_matrix = back_end.measure_hdmm(workload=hist, eps=epsilon, restarts=20)

    prev_spec = current_spec_arr[id]
    # print(prev_spec)
    prev_spec.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)

    current_spec_arr[id], cached = back_end.display(hist) ################# Cache_search function called

    spec = current_spec_arr[id].join(prev_spec, on=[left_col[0], right_col[0]]).reset_index([left_col[0], right_col[0]])

    with plot_output_arr[id]:
        plot_output_arr[id].clear_output()
        plot = src.plots.linked_hist_test(left_col[0], right_col[0], data=spec, projection=True,label=False)
        display(plot)

    hist_plot_rm = widgets.VBox([plot_output_arr[id]])
    display(hist_plot_rm)

    # for i in range(len(questions) - 2):
    #     key_val2 = [x for x in visualizations[i + 2].items()]
    #     measure_dict2 = {'left': key_val2[0], 'right': key_val2[1]}
    #     left_col2 = measure_dict2['left']
    #     right_col2 = measure_dict2['right']
    #     widths = {left_col2[0]:left_col2[1], right_col2[0]:right_col2[1]}
    #
    #     prev_spec2 = current_spec_arr[i + 1]
    #     prev_spec2.rename(columns={'error': 'error_prev', 'plus_error': 'plus_error_prev', 'minus_error': 'minus_error_prev', 'true_count':'true_count_prev', 'noisy_count':'noisy_count_prev'}, inplace=True)
    #     # hist2 = hist_workloads[i + 2]
    #     hist2 = builder.histogram_workload(cps_domain.config, bin_widths=widths)
    #
    #     current_spec_arr[i + 1], cached2 = back_end.display(hist2) ################# Cache_search function called
    #
    #     spec2 = current_spec_arr[i + 1].join(prev_spec2, on=[left_col2[0], right_col2[0]]).reset_index([left_col2[0], right_col2[0]])
    #     spec2 = spec2.round(0)
    #
    #     with plot_output:
    #         # plot_output.clear_output()
    #         plot2 = src.plots.linked_hist_test(left_col2[0], right_col2[0], data=spec2, projection=True,label=False)
    #         # plot = src.plots.linked_hist(left_col[0], right_col[0], data=spec, display_true=False, history=False)
    #         display(plot2)
    
box_layout = widgets.Layout(display='flex',
    flex_flow='column',
    align_items='flex-start',
    color='black',
    width='80%')

# bar_label = widgets.HBox([budget_spent, progress_bar, make_it_better])
prompt_answer = widgets.VBox([answer, submit_btn], layout=box_layout)

In [18]:
# display(bar_label)
interface = widgets.VBox([prompt, prompt_answer])
display(interface)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

Button(description='Submit', layout=Layout(height='50px', width='20%'), style=ButtonStyle())

In [19]:
# print(questions)